In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,auc,classification_report,confusion_matrix,mean_squared_error, precision_score, recall_score,roc_curve

import time
from numpy import absolute
import joblib
from datetime import datetime, timedelta

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days-of-ml/train.csv
/kaggle/input/30-days-of-ml/test.csv


### Load data

In [2]:
# Load and review train data
train_data = pd.read_csv("/kaggle/input/30-days-of-ml/train.csv", low_memory=False, index_col='id')
test_data = pd.read_csv("/kaggle/input/30-days-of-ml/test.csv", low_memory=False, index_col='id')
#submission_example = pd.read_csv("/kaggle/input/30-days-of-ml/sample_submission.csv", low_memory=False)

train_data.describe()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,0.527335,0.460926,0.490498,0.496689,0.491654,0.510526,0.467476,0.537119,0.498456,0.474872,0.474492,0.473216,0.494561,0.508273,8.241979
std,0.230599,0.214003,0.253346,0.219199,0.240074,0.228232,0.210331,0.218140,0.239920,0.218007,0.255949,0.222022,0.247292,0.222950,0.746555
min,-0.118039,-0.069309,-0.056104,0.130676,0.255908,0.045915,-0.224689,0.203763,-0.260275,0.117896,0.048732,0.052608,-0.074208,0.151050,0.140329
25%,0.405965,0.310494,0.300604,0.329783,0.284188,0.354141,0.342873,0.355825,0.332486,0.306874,0.276017,0.308151,0.289074,0.300669,7.742071
50%,0.497053,0.427903,0.502462,0.465026,0.390470,0.488865,0.429383,0.504661,0.439151,0.434620,0.459975,0.433812,0.422887,0.472400,8.191373
75%,0.668060,0.615113,0.647512,0.664451,0.696599,0.669625,0.573383,0.703441,0.606056,0.614333,0.691579,0.642057,0.714502,0.758447,8.728634
max,1.058443,0.887253,1.034704,1.039560,1.055424,1.067649,1.111552,1.032837,1.040229,0.982922,1.055960,1.071444,0.975035,0.905992,10.411992


### Features engineering

In [3]:
# Separate target from predictors
y = train_data.target
features = train_data.drop(['target'], axis=1) # Remove target

# List of features for later use
feature_list = list(features.columns)

# Preview features
features.head()

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

X = features.copy()
X_test = test_data.copy()

# Separate categorical and numerical columns (all categorical)
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 20 and 
                        X[cname].dtype == "object"]
print("Categorical columns: ", categorical_cols)

ordinal_encoder = OrdinalEncoder()
X[categorical_cols] = ordinal_encoder.fit_transform(features[categorical_cols])
X_test[categorical_cols] = ordinal_encoder.transform(test_data[categorical_cols])

print("Train data after transform:")
print(X.head())

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, 
                                                                test_size=0.2,random_state=42)

print("X_train shape: ",X_train.shape)
print("X_valid shape: ",X_valid.shape)
print("y_train shape: ",y_train.shape)
print("y_valid shape: ",y_valid.shape)


Categorical columns:  ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
Train data after transform:
    cat0  cat1  cat2  cat3  cat4  cat5  cat6  cat7  cat8  cat9  ...     cont4  \
id                                                              ...             
1    1.0   1.0   1.0   2.0   1.0   1.0   0.0   4.0   2.0  13.0  ...  0.610706   
2    1.0   1.0   0.0   0.0   1.0   3.0   0.0   5.0   0.0  14.0  ...  0.276853   
3    0.0   0.0   0.0   2.0   1.0   3.0   0.0   3.0   0.0   5.0  ...  0.285074   
4    1.0   1.0   0.0   2.0   1.0   3.0   0.0   4.0   2.0  10.0  ...  0.284667   
6    0.0   0.0   0.0   2.0   1.0   3.0   0.0   4.0   0.0  13.0  ...  0.287595   

       cont5     cont6     cont7     cont8     cont9    cont10    cont11  \
id                                                                         
1   0.400361  0.160266  0.310921  0.389470  0.267559  0.237281  0.377873   
2   0.533087  0.558922  0.516294  0.594928  0.341439  0.906013  0.921701 

### 

### Define model, train, test and predict

In [4]:
xgb_params = {'objective': 'reg:squarederror',
              'n_estimators': 10000,
              'learning_rate': 0.036,
              'subsample': 0.926,
              'colsample_bytree': 0.118,
              'grow_policy':'lossguide',
              'max_depth': 4,
              'booster': 'gbtree', 
              'reg_lambda': 45.1,
              'reg_alpha': 34.9,
              'random_state': 42,
              'reg_lambda': 0.00087,
              'reg_alpha': 23.13181079976304,
              'n_jobs': 4,
              'tree_method': 'gpu_hist'}

xgb_model = XGBRegressor(**xgb_params)
#xgb_model.get_params()

start = time.time()
xgb_model.fit(X_train, y_train,verbose=False)
stop = time.time()
print("Training time = ", timedelta(seconds=int(stop-start)))

# Preprocessing of validation data, get predictions
preds_valid = xgb_model.predict(X_valid)
mse_score=mean_squared_error(y_valid, preds_valid, squared=False)
print("Train score MSE", mse_score)

# Use the model to generate predictions
predictions = xgb_model.predict(X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})
output.to_csv('xgboost_only_cat_ord_enc_2.csv', index=False)


#Plot 
#plt.rcParams["axes.labelsize"] = 12
#xgb_prob_train = xgb_model.predict(X_train) - y_train
#plt.figure(figsize=(6,6))
#sp.stats.probplot(xgb_prob_train, plot=plt, fit=True)
#plt.title('Train Probability Plot for XGBoost', fontsize=10)
#plt.show()

#xgb_prob_test = xgb_model.predict(X_valid) - y_valid
#plt.figure(figsize=(6,6))
#sp.stats.probplot(xgb_prob_test, plot=plt, fit=True)
#plt.title('Test Probability Plot for XGBoost', fontsize=10)
#plt.show()


Training time =  0:00:27
Train score MSE 0.717213069826007
